# FreeMonad

# 何がFreeなのか?
Freeモナドは特徴として、`Fucntor`から`Monad`を生成する事が出来ます。そして、その`Functor`の具象に応じた特性をもった`Monad`が生成されます。
[こちらの記事](https://qiita.com/hiruberuto/items/8bbc0343bf794c368287)でポケモンに例えるなら、
> このモナドはポケモンで言うとメタモンのような、それ自身では何もできないが、どんなモナドにも変化することができるという、とにかくメタメタしい性質を持っています

メタモン。。。たしかにメタモンのようになんというか、つかみ辛い。。。
さらに、[こちらの記事](http://konn-san.com/prog/never-understood-monad-tutorial.html])には

> モナドは合成可能な命令書であること
自由モナドを用いたプログラミングは、命令書を組み立てて、それを後から実行することに相当する。
実は、全てのモナドは自由モナドをつくって、そいつを適当に「実行」することで作ることが出来る
自由モナドがプログラムのソースコードで、それを実行する「処理系」を色々に書くことで色んな計算が出来る！インタプリタを書くようなもの！
モナドは怖くなかった！

おおっ！強そう。`Functor`から**さまざまな特性をもったMonadを自由に(それこそメタモンのように)つくれることが`Free`の由来ではないかと思いました。**  
なので、ここから、具象のFuncotorを定義し、DIするまでの具体例とともにつらづら書いていきます

In [ ]:
## レシピ

In [ ]:
1 Functorを定義する

In [1]:
sealed trait Functor[F[_]]{
    def map[A,B](t:F[A])(f:A=>B):F[B]
}

defined trait Functor

In [ ]:
Functorとは以下の要件をみたすものです

In [ ]:
- mapのfにx => xを入れて生成されたものが、元の値と等しい
assert( t.map(x => x) == t )
- 適当な関数gとhについて、gとhの合成関数（x => g(h(x))）でmapした値と、hでmapした値をgでmapした値が等しい
assert( t.map(x => g(h(x))) == t.map(h).map(g) )

# 以下メモ

In [15]:
trait Functor[F[_]] {
  def map[A, B](m: F[A])(f: A => B): F[B]
}

defined trait Functor

次にFreeMonadの基盤を用意

In [16]:
sealed trait FreeM[S[+_], +A] {
  def flatMap[B](f: A => FreeM[S, B])(implicit s: Functor[S]): FreeM[S, B]
  def map[B](f: A => B)(implicit s: Functor[S]): FreeM[S, B] = flatMap(a => Pure(f(a)))
}

case class Pure[S[+_], +A](a: A) extends FreeM[S, A] {
  def flatMap[B](f: A => FreeM[S, B])(implicit s: Functor[S]): FreeM[S, B] = f(a)
}

case class Free[S[+_], +A](k: S[FreeM[S, A]]) extends FreeM[S, A]{
  def flatMap[B](f: A => FreeM[S, B])(implicit s: Functor[S]): FreeM[S, B] = Free(s.map(k)(_.flatMap(f)))
}

defined trait FreeM
defined class Pure
defined class Free

ここから具体例。  
なんらかのインターフェースに一文字ずつ入力する`getChar`メソッドと一文字ずつ出力する`putChar`メソッドを定義したい。ただし、この二つのメソッドは**あらゆるインターフェースに対して汎用的な形で定義したい**。つまり、入力出力と、そのための???

In [17]:
// どうしよう???
def getCh= ???
def putCh(ch:Char)= ???

defined function getCh
defined function putCh

In [18]:
sealed trait CharIO[+A]
case class GetCh[+A](f:Char => A) extends CharIO[A]
case class PutCh[+A](c:Char, a:A) extends CharIO[A]

defined trait CharIO
defined class GetCh
defined class PutCh

In [19]:
implicit val charIOFunctor =
  new Functor[CharIO] {
    def map[A, B](a: CharIO[A])(f: A => B): CharIO[B] = a match {
      case GetCh(g)    => GetCh(f compose g)
      case PutCh(c, x) => PutCh(c, f(x))
    }
  }

charIOFunctor: AnyRef with Functor[CharIO] = cmd18$$user$$anonfun$1$$anon$1@49abb390

In [20]:
def getCh:FreeM[CharIO, Char] = Free(GetCh({ch => Pure(ch)}))
def putCh(ch:Char):FreeM[CharIO, Unit] = Free(PutCh(ch, Pure(())))

defined function getCh
defined function putCh

In [30]:
def mapFreeM[S[+_]:Functor, A](f:A => FreeM[S,Unit], s:Seq[A]):FreeM[S,Unit] = s.toList match {
  case x::xs => xs.foldLeft(f(x)){(m:FreeM[S,Unit],c:A) => m.flatMap{unit => f(c)} }
  case Nil => Pure(())
}

val io:FreeM[CharIO, Unit] = for{
    _  <- mapFreeM(putCh, "Hello, Scala Programmer! Please input a character:")
    ch <- getCh
    _  <- mapFreeM(putCh, "The ordinal of the character is:")
    _  <- mapFreeM(putCh, ch.toInt.toString)
    _  <- mapFreeM(putCh, ".\n Thank you!\n")
  } yield ()

defined function mapFreeM
io: FreeM[CharIO, Unit] = Free(PutCh(H,Free(PutCh(e,Free(PutCh(l,Free(PutCh(l,Free(PutCh(o,Free(PutCh(,,Free(PutCh( ,Free(PutCh(S,Free(PutCh(c,Free(PutCh(a,Free(PutCh(l,Free(PutCh(a,Free(PutCh( ,Free(PutCh(P,Free(PutCh(r,Free(PutCh(o,Free(PutCh(g,Free(PutCh(r,Free(PutCh(a,Free(PutCh(m,Free(PutCh(m,Free(PutCh(e,Free(PutCh(r,Free(PutCh(!,Free(PutCh( ,Free(PutCh(P,Free(PutCh(l,Free(PutCh(e,Free(PutCh(a,Free(PutCh(s,Free(PutCh(e,Free(PutCh( ,Free(PutCh(i,Free(PutCh(n,Free(PutCh(p,Free(PutCh(u,Free(PutCh(t,Free(PutCh( ,Free(PutCh(a,Free(PutCh( ,Free(PutCh(c,Free(PutCh(h,Free(PutCh(a,Free(PutCh(r,Free(PutCh(a,Free(PutCh(c,Free(PutCh(t,Free(PutCh(e,Free(PutCh(r,Free(PutCh(:,Free(GetCh(<function1>))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))

In [36]:
def runCharIO(free:FreeM[CharIO, Unit]):Unit = free match { 
  case Pure(a)               => a
  case Free(GetCh(f))        => runCharIO(f(readChar()))
  case Free(PutCh(ch, cont)) => print(ch);runCharIO(cont)
}

defined function runCharIO

In [36]:
runCharIO(io)

: 

## 何が自由なのか
[こちらの記事に](http://d.hatena.ne.jp/fumiexcel/20121111/1352614885)

> Freeモナドのメリットは、諸概念を限定された世界に閉じ込めて抽象化できることだけではない。自分でMonadを作ると、MonadWriterなどの型クラスの大量のインスタンス宣言をしなければならないが、Freeモナドならその必要はない。Freeモナドの性質を決めるのはあくまでFunctorなので、そういったインスタンス宣言をFreeそのものに集約できる。いわば、モナドがタダ(free)で手に入るのだ。

とある
わかるようなわからないような。。。

# FreeMonadを知った上で今後やりたいこと
- データ分析のフローとロジックをFreeMonadの概念を応用してライブラリを作りたい

# リファレンス
- ほとんど読んでないですが、[Functional and Reactive Domain Modeling](https://www.amazon.co.jp/Functional-Reactive-Domain-Modeling-Debasish/dp/1617292249)の五章にもFreeMonadについて書いてあります
- 同じくざっとしか読んでないですが、[Scala HighPerformance Programing](https://www.amazon.co.jp/dp/B01BKL1PGA/ref=dp-kindle-redirect?_encoding=UTF8&btkr=1)の七章にもパフォーマンス観点で書かれてます

